# Stage 1: Import Everything

In [1]:
import numpy as np
import math
import re
import time
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
import xml.etree.ElementTree as ET


# Stage 2: Data preprocessing

In [2]:
#run this if you want to find out what causes an error, just change the index
indexWithError = 190361
with open("CCMatrix v1- EN to AR Dataset.tmx", mode='r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        if i == indexWithError:  # because line numbers start from 0
            print(line)
            break

      <tuv xml:lang="en"><seg>Watch The Squid and the Whale now on MUBI.</seg></tuv>


In [3]:
def clean_control_characters(line):
    # Remove control characters except for tab, newline, and carriage return
    return re.sub(r'[\x00-\x08\x0b\x0c\x0e-\x1f]', '', line)

In [4]:

start_time = time.time()

with open("CCMatrix v1- EN to AR Dataset.tmx", mode='r', encoding='utf-8') as f:
    lines = f.readlines()

cleaned_lines = [clean_control_characters(line) for line in lines]

with open("CCMatrix v1- EN to AR Dataset.tmx", mode='w', encoding='utf-8') as f:
    f.writelines(cleaned_lines)

end_time = time.time()

time_taken = end_time - start_time
print(f"Time taken to process the file: {time_taken:.2f} seconds")

Time taken to process the file: 959.04 seconds


In [3]:
import xml.etree.ElementTree as ET

ar_texts = []
en_texts = []
counter = 0

# Create a dictionary for namespaces
namespaces = {'xml': 'http://www.w3.org/XML/1998/namespace'}

with open("CCMatrix v1- EN to AR Dataset.tmx", mode='r', encoding='utf-8') as f:
    # Parse the content into an XML tree
    tree = ET.parse(f)
    root = tree.getroot()
    counter = 0
    for tu in root.findall('.//tu'):
        current_tu = {}

        for tuv in tu.findall('tuv'):
            lang = tuv.get("{http://www.w3.org/XML/1998/namespace}lang")
            seg_text = tuv.find('seg').text
            if seg_text:  # Check if seg_text is not None
                current_tu[lang] = clean_control_characters(seg_text)

        if "ar" in current_tu and "en" in current_tu:
            ar_texts.append(current_tu["ar"])
            en_texts.append(current_tu["en"])

        counter += 1
        if counter == 5:
            break

# print to check results
print("Arabic:", ar_texts)
print("English:", en_texts)


ParseError: not well-formed (invalid token): line 190362, column 36 (<string>)

In [ ]:
tokenizer = Tokenizer(num_words=2 ** 16, oov_token='<OOV>')
#tokenizer.fit_on_texts(data_clean)